In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [18]:
# 데이터 로드
article_df = pd.read_csv("sample2.csv")
len(article_df)

29

In [19]:
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         29 non-null     object
 1   title       29 non-null     object
 2   date        29 non-null     object
 3   youtube_id  29 non-null     object
 4   comment     29 non-null     object
 5   like_num    29 non-null     object
 6   class       29 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 1.7+ KB


In [23]:
article_df['class']

0     1
1     1
2     1
3     1
4     0
5     0
6     1
7     1
8     1
9     0
10    0
11    2
12    1
13    1
14    0
15    1
16    0
17    2
18    1
19    1
20    0
21    2
22    2
23    0
24    0
25    0
26    2
27    0
28    1
Name: class, dtype: int64

In [24]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
article_df.comment, article_df['class'], test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(26, 3, 26, 3)

In [25]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [26]:
# 학습
model = clf.fit(X_train.values.astype("str"), y_train)

In [27]:
# 모델 성능 확인
y_pred = model.predict(X_test)

In [28]:
list(y_test[:5]), list(y_pred[:5])

([0, 2, 1], [0, 1, 1])

In [29]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         1
          1       0.50      1.00      0.67         1
          2       0.00      0.00      0.00         1

avg / total       0.50      0.67      0.56         3



/home/loveactualry/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### 모델 사용하기

In [30]:
categories = {
    0: "택시",
    1: "타다",
    2: "미분류",

}

In [31]:
contents = [
    "결국 모빌리티하려면 택시사업을 하라는거네ㅋㅋㄹ 택시회사를 세우란건가? 사납금으로 운영하는?",
    "저는 택시와 관련 없는 사람입니다 다만 타다라는 업태 형때가 운수업 이다보니 택시업 관광버스 업자와 겹처 있는 형태 입니다 당연히 택시와 버스사업자 들의 피해가 발생 할수있다 볼수 있습니다 운수업 등록하셔서 정식으로 영업하시는 것이 옳은  것이라 생각 합니다",
    "택시 없어지고 타다 대중화대서 대중교통 편리성으로 되라고 응원하는분들 어디갓지 조용하네 타다는 없어져야되",
]

In [32]:
datas = {
    "content": contents,
    "category_code": model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,결국 모빌리티하려면 택시사업을 하라는거네ㅋㅋㄹ 택시회사를 세우란건가? 사납금으로 운...,1
1,저는 택시와 관련 없는 사람입니다 다만 타다라는 업태 형때가 운수업 이다보니 택시업...,0
2,택시 없어지고 타다 대중화대서 대중교통 편리성으로 되라고 응원하는분들 어디갓지 조용...,0


In [33]:
round(max(model.predict_proba([contents[0]])[0]), 2)

0.46

In [34]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(
    lambda data: round(max(model.predict_proba([data])[0]), 2) )
df

,content,category_code,category,proba
0,결국 모빌리티하려면 택시사업을 하라는거네ㅋㅋㄹ 택시회사를 세우란건가? 사납금으로 운...,1,타다,0.46
1,저는 택시와 관련 없는 사람입니다 다만 타다라는 업태 형때가 운수업 이다보니 택시업...,0,택시,0.97
2,택시 없어지고 타다 대중화대서 대중교통 편리성으로 되라고 응원하는분들 어디갓지 조용...,0,택시,0.62


In [35]:
# 모델 저장

In [36]:
pickle.dump(model, open("clf.pkl", "wb"))